In [1]:
from datasets import *
from train import *
from torchvision import datasets, transforms, models

training

In [2]:
teacher = models.efficientnet_b0(weights='DEFAULT').to('cuda').eval()
student = models.shufflenet_v2_x0_5(weights='DEFAULT').to('cuda').eval()

In [3]:
with torch.no_grad():
    # first reinitialize the layer before classification to match the teacher feature dimensions
    student.conv5[0] = torch.nn.Conv2d(student.conv5[0].in_channels,teacher.classifier[1].in_features,kernel_size=(1, 1), stride=(1, 1), bias=False)
    student.conv5[1] = torch.nn.BatchNorm2d(teacher.classifier[1].in_features,eps=1e-05,momentum=0.1,affine=True,track_running_stats=True)

    # next create a new fc layer to match the teacher dimension
    student.fc = torch.nn.Linear(teacher.classifier[1].in_features,teacher.classifier[1].out_features)

    # finally copy the teacher fc parameters
    student.fc.weight[:,:] = teacher.classifier[1].weight[:,:]
    student.fc.bias[:] = teacher.classifier[1].bias[:]

    # freeze the classification layer
    for param in student.fc.parameters():
        param.requires_grad = False

    student.to('cuda')

In [5]:
train_loader, val_loader = load_imagenet(128,12345)
train(student,train_loader,val_loader,'cuda',lr=0.004,epochs=100,log_name="sn_random_head")

Train Epoch: 0 [0/1153050 (0%)] train loss: 3.049, lr: 0.00400000
Train Epoch: 0 [102400/1153050 (9%)] train loss: 2.721, lr: 0.00400000
Train Epoch: 0 [204800/1153050 (18%)] train loss: 2.963, lr: 0.00398000
Train Epoch: 0 [307200/1153050 (27%)] train loss: 2.291, lr: 0.00396010
Train Epoch: 0 [409600/1153050 (36%)] train loss: 2.644, lr: 0.00394030
Train Epoch: 0 [512000/1153050 (44%)] train loss: 2.461, lr: 0.00392060
Train Epoch: 0 [614400/1153050 (53%)] train loss: 2.875, lr: 0.00390100
Train Epoch: 0 [716800/1153050 (62%)] train loss: 2.377, lr: 0.00388149


KeyboardInterrupt: 

testing

In [40]:
test_loader = load_imagenet(128,1234,False)

In [12]:
torch.load("best_batch_i288264sn_frozen_head_long1682578354.1065493.pth")['lr']

[5.142319898206824e-06]

In [14]:
student = models.shufflenet_v2_x0_5(weights='DEFAULT').to('cuda')
with torch.no_grad():
    # first reinitialize the layer before classification to match the teacher feature dimensions
    student.conv5[0] = torch.nn.Conv2d(student.conv5[0].in_channels,teacher.classifier[1].in_features,kernel_size=(1, 1), stride=(1, 1), bias=False)
    student.conv5[1] = torch.nn.BatchNorm2d(teacher.classifier[1].in_features,eps=1e-05,momentum=0.1,affine=True,track_running_stats=True)

    # next create a new fc layer to match the teacher dimension
    student.fc = torch.nn.Linear(teacher.classifier[1].in_features,teacher.classifier[1].out_features)

    # freeze the classification layer
    for param in student.fc.parameters():
        param.requires_grad = False
student.load_state_dict(torch.load("best_batch_i288264sn_frozen_head_long1682578354.1065493.pth")['model_state_dict'])
student.to('cuda')
student.train()

ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_

In [42]:
acc = validate(student,test_loader,'cuda')

100%|██████████| 391/391 [04:09<00:00,  1.57it/s]


In [43]:
acc

(0.50082, 2.231632464986933, 0.7435)